setup notebook

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
# buka file dataset dari gdrive
from google.colab import drive
drive.mount('/content/gdrive')

link_folder = '/content/gdrive/MyDrive/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


EDA (Exploratory Data Analysis)


In [91]:
"""
references:
- https://towardsdatascience.com/community-detection-algorithms-9bd8951e7dae
- https://medium.com/@marcosacj/creating-and-visualizing-a-complex-network-of-instagram-hashtags-based-on-posts-about-politics-2daf24f31088
- https://github.com/marcosacj/cna-instagram/blob/master/2019-06-11-1-macj-graph-scratch.ipynb
- https://www.kaggle.com/datasets/arunavakrchakraborty/covid19-twitter-dataset
"""

'\nreference:\n- https://towardsdatascience.com/community-detection-algorithms-9bd8951e7dae\n- https://github.com/marcosacj/cna-instagram/blob/master/2019-06-11-1-macj-graph-scratch.ipynb\n- https://www.kaggle.com/datasets/arunavakrchakraborty/covid19-twitter-dataset\n\n'

new covid 19 twitter dataset from a

In [92]:
df = pd.read_csv( link_folder+'Covid-19 Twitter Dataset (Apr-Jun 2020).csv' )
# df = pd.read_csv( link_folder+'Covid-19 Twitter Dataset (Apr-Jun 2021).csv' )
filtered_df = df[ df['hashtags'].notnull() ].copy()

# # check unique values from attributes from df
# print( "Unique values from category column: ", sorted( df['hashtags'].unique() ), ", count: ", df['hashtags'].nunique() )

# preview data from file
# df.head()
print( "origin data number: ", df.shape[0] )
print( "filtered data number: ", filtered_df.shape[0] )

filtered_df.info()

origin data number:  143903
filtered data number:  28510
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28510 entries, 4 to 143899
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               28510 non-null  float64
 1   created_at       28510 non-null  object 
 2   source           28500 non-null  object 
 3   original_text    28510 non-null  object 
 4   lang             28510 non-null  object 
 5   favorite_count   28510 non-null  float64
 6   retweet_count    28510 non-null  float64
 7   original_author  28510 non-null  object 
 8   hashtags         28510 non-null  object 
 9   user_mentions    20999 non-null  object 
 10  place            21231 non-null  object 
 11  clean_tweet      28454 non-null  object 
 12  compound         28510 non-null  float64
 13  neg              28510 non-null  float64
 14  neu              28510 non-null  float64
 15  pos              28510 non-null  float64
 16  

In [93]:
hashtags_on_posts = filtered_df['hashtags'].unique()
number_current_hastags_pairs = filtered_df['hashtags'].nunique()

# get list of one hashtags from posts
def get_hashtags_list( posts ):
  # initial value
  final_list_tags = []
  final_list_posts = []
  max_posts = 100

  # loop tiap satu data dari hashtags, misal limit dulu ke 1000 post
  for all_hashtags in posts[ :max_posts ]:
    # divide hashtags into one list
    array_hashtags = all_hashtags.lower().split( "," )

    # save current hastags list from one post
    final_list_posts.append( array_hashtags )

    # loop tiap hashtags
    for hash in array_hashtags:
      # normalsasi penulisan hashtags dalam bentuk lowercase untuk meminimalkan data double
      if hash.lower() not in final_list_tags: final_list_tags.append( hash.lower() )
  # end loop

  # return final list
  return final_list_tags, final_list_posts
# end function

# testing function
# varitions_of_hashtags, list_of_hashtags_from_each_post = get_hashtags_list( hashtags_on_posts[0:100] )
varitions_of_hashtags, list_of_hashtags_from_each_post = get_hashtags_list( hashtags_on_posts )
print( varitions_of_hashtags )
print( 'banyak jenis hashtags: ', len( varitions_of_hashtags ) )
print( 'banyak posts: ', len( list_of_hashtags_from_each_post ) )

['covid_19', 'coronavirus', 'covid19', 'oic', ' islamophobic', ' india', ' covid', 'quarantinewine', 'abhealth', ' covid19ab', ' medicareforall', 'openamerica', 'frontlineheroes', 'togetherathome', 'covid', 'worthlessapologies', 'backboris', 'plasma', ' hmc', ' yoursafetyismysafety', ' coronavirus', ' covid_19', ' qatar', 'physicaldistancing', ' selfisolation', 'trump', ' timscott', '60minutes', ' covid19', ' environment', 'live', 'saheart', ' movemoremonth', ' movemore', 'howwefeel', ' demcastoh', 'cityofto', 'covid__19', ' cuba', 'blackaf', 'lagos', 'cloud28', ' covid__19', ' china', 'crypto', ' malware', ' cybersecurity', ' infosec', ' infosecurity', 'singapore', 'peanutbutter', ' asymptomatic', ' carriers', ' covidtesting', ' testing', ' senseofsmell', 'wuhan', 'lagosfoodrelief', 'setumerabodyguard', ' stayhome', 'cubasalvavidas', ' somoscuba', ' somoscontinuidad', 'fisa', ' r0', 'unibenvsunilag', ' ramadan', 'lockdown', 'np', 'coronaviruspandemic', ' coronavirusoutbreak', 'covidー1

In [94]:
# create pair of two string
def create_string_pair( selected_value, other_values ):
  return [ ( selected_value, x ) for x in other_values[1:] ]
# end func

# get list of frequencies number between two hashtags
def get_hashtags_frequencies( posts, hashtags_list ):
  # initial value
  final_pair_list = [] # with all created pairs
  final_freq_list = [] # with all frequencies of each pair list
  final_list_2 = [] # with index `hash 1` and values `hash 2, hash 3, hash 4, dst`

  # create hashtags pair between selected pair and remaining hashtags
  for i in range( len( hashtags_list ) ):
    # print( i )
    # bikin pasangannya
    pairs = create_string_pair( hashtags_list[ i ], hashtags_list[i+1: ] )

    # loop
    for one_pair in pairs:
      # divide both hashtags
      hash_1 = one_pair[0]
      hash_2 = one_pair[1]

      # loop posts contains hashtags list
      freq_counter = 0
      for one_post in posts:
        if ( hash_1 in one_post ) and ( hash_2 in one_post ):
          freq_counter += 1

          # # test
          # print( one_post, ", -> ", hash_1, " AND ", hash_2 )
        # end if
      # end loop posts

      # only save freq with exist hashtags pairs
      if freq_counter > 0:
        final_pair_list.append( one_pair )
        final_freq_list.append( freq_counter )
      # end if

    # end loop pairs

  # end loop range list

  # return final list
  return final_pair_list, final_freq_list
# end function

def get_hashtags_pair_lists( posts, hashtags_list ):
  # initial value
  final_pair_list = [] # with all created pairs

  # create hashtags pair between selected pair and remaining hashtags
  for i in range( len( hashtags_list ) ):
    # bikin pasangannya
    pairs = create_string_pair( hashtags_list[ i ], hashtags_list[i+1: ] )

    # loop
    for one_pair in pairs:
      # divide both hashtags
      hash_1 = one_pair[0]
      hash_2 = one_pair[1]

      # loop posts contains hashtags list
      # only save hashtags pairs that exists in posts
      for one_post in posts:
        if one_pair not in final_pair_list and ( hash_1 in one_post ) and ( hash_2 in one_post ):
          final_pair_list.append( one_pair )
        # end if
      # end loop posts

    # end loop pairs

  # end loop range list

  # return final list
  return final_pair_list
# end function

# hashtags_on_posts --> hashtags data in form of string from each posts
# varitions_of_hashtags --> hashtags variety from all posts
# list_of_hashtags_from_each_post --> each hashtags list on each posts

# list_edge_between_nodes, list_edges_weight = get_hashtags_frequencies( list_of_hashtags_from_each_post, varitions_of_hashtags )
# print( "list_edge_between_nodes: ", list_edge_between_nodes )
# print( "list_edges_weight: ", list_edges_weight )
# print( "length: ", len( list_edges_weight ) )


list_edge_between_nodes = get_hashtags_pair_lists( list_of_hashtags_from_each_post, varitions_of_hashtags )
print( "list_edge_between_nodes: ", list_edge_between_nodes )

list_edge_between_nodes:  [('covid_19', ' malware'), ('covid_19', ' cybersecurity'), ('covid_19', ' infosec'), ('covid_19', ' infosecurity'), ('covid_19', ' r0'), ('covid_19', ' heartmath'), ('covid_19', ' coherence'), ('coronavirus', ' covid19'), ('coronavirus', ' environment'), ('coronavirus', ' brazil'), ('covid19', ' medicareforall'), ('covid19', ' coronavirus'), ('covid19', ' demcastoh'), ('covid19', ' stayhome'), ('oic', ' india'), ('oic', ' covid'), (' islamophobic', ' covid'), (' india', ' covid_19'), (' india', ' stayhome'), (' india', 'singer'), (' india', ' song'), (' india', ' entertainment'), (' india', ' perform'), (' india', ' love'), (' india', ' people'), (' india', ' lockdown'), (' covid', ' covid19'), (' covid', 'live'), (' covid', 'cryptofunds'), (' covid', 'nyc'), (' covid', ' barber'), (' covid', ' relief'), (' covid', ' hedliash'), (' covid', ' fundingtweets'), (' covid', ' support'), (' covid', ' gofundme'), (' covid', ' plz'), (' covid', 'oxygen'), ('plasma', '